In [211]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
from datetime import datetime, timezone, timedelta
import logging
import pickle

logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format for the log messages
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

%reload_ext autoreload
%autoreload 2
from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from config.constants import *
from data.processed.dataset_creation import DatasetCreator
from data.processed.indicators import *
from data.processed.targets import Balanced3ClassClassification
from data.processed.normalization import ZScoreOverWindowNormalizer, ZScoreNormalizer, MinMaxNormalizer
from data.processed.missing_values_handling import DummyMissingValuesHandler
from data.processed.dataset_pytorch import DatasetPytorch
from data.raw.retrievers.alpaca_markets_utils import get_stock_stats
from modeling.trainer import Trainer
from observability.mlflow_integration import log_experiment

from config.experiments.cur_experiment import config

In [141]:
retriever = AlpacaMarketsRetriever()
all_symbols = retriever.get_all_symbols()
len(all_symbols)

3991

In [142]:
START_DATE = "2025-06-15 16:00:00+00:00"
END_DATE = "2025-07-01 16:00:00+00:00"
stock_stats = get_stock_stats(retriever, all_symbols, START_DATE, END_DATE)

with open(f'stock_stats_06.pkl', 'wb') as f:
    pickle.dump(stock_stats, f)

Processing (0/3991)
Processing (10/3991)


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing (20/3991)


c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\retrievers\alpaca_markets_utils.py:72: RuntimeWarning: Mean of empty slice
  volatility_1h = np.nanmean(vol_1h_list)


Processing (30/3991)
Processing (40/3991)
Processing (50/3991)
Processing (60/3991)


...

Processing (3960/3991)
Processing (3970/3991)


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing (3980/3991)


c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ikurnosau\anaconda3\envs\mldl\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing (3990/3991)


In [10]:
volatility_5m_to_spread = {symbol: stock_stats[symbol]['volatility_5m'] / stock_stats[symbol]['avg_spread'] for symbol in stock_stats}
best_stocksed_5m = sorted(volatility_5m_to_spread.items(), key=lambda x: x[1], reverse=True)[:100]
best_stocksed_5m

[('QQQ', 40.83391846772082),
 ('RAPT', 37.81413590471789),
 ('SPY', 37.07878330689369),
 ('IBKR', 31.174461809791328),
 ('NKE', 28.564417408805188),
 ('APG', 28.34896193304662),
 ('VERV', 27.7428913840944),
 ('PLTR', 26.812425853496155),
 ('GOOG', 26.564549976815325),
 ('NVDA', 26.16204246932238),
 ('AMD', 24.50965934591024),
 ('AAPL', 24.22285779937311),
 ('GLD', 22.384304976575894),
 ('IWM', 22.06688854883784),
 ('GOOGL', 21.729012918724504),
 ('TQQQ', 20.783871852865126),
 ('FTV', 20.740948093783363),
 ('TSLA', 20.234120409614366),
 ('SPTN', 20.22819782689736),
 ('SMH', 20.116810402375805),
 ('XLY', 18.738691348601876),
 ('HIMS', 18.20337969339497),
 ('XLK', 18.03711955546672),
 ('MRVL', 17.97620495552505),
 ('UBER', 17.404823707043303),
 ('XOM', 16.513753946186903),
 ('HOOD', 16.383302735653157),
 ('AMZN', 16.343849814811318),
 ('RKLB', 16.239410440426862),
 ('PEGA', 16.020436971451122),
 ('ENPH', 15.708249673403035),
 ('GBTC', 15.547399408888525),
 ('GDXJ', 15.280855871969973),
 (